<a href="https://colab.research.google.com/github/xinyueli2896/RapRec/blob/main/main_Demucs%2BWav2Vec2Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning Speech Model with 🤗 Transformers**

In [ ]:
import librosa
import torch
import transformers
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

c:\Users\wbsgus\anaconda3\envs\rap\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare Data, Tokenizer, Feature Extractor

### Create Wav2Vec2CTCTokenizer

In [ ]:
from datasets import load_dataset, load_metric
dataset_path = "dataset_cleaned_sep"
timit = load_dataset("audiofolder", data_dir=dataset_path)

Resolving data files: 100%|██████████| 3085/3085 [00:00<00:00, 39181.54it/s]


In [ ]:
timit

DatasetDict({
    train: Dataset({
        features: ['audio', 'lyrics'],
        num_rows: 12532
    })
    test: Dataset({
        features: ['audio', 'lyrics'],
        num_rows: 3085
    })
})

In [ ]:
# timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [ ]:
print(timit['train'][0])
type(timit['train'])

{'audio': {'path': 'E:/rap_rec/dataset_cleaned_sep/train/chunk_0.mp3', 'array': array([ 0.00000000e+00,  1.46443085e-09,  1.37479139e-09, ...,
       -3.44191722e-05, -3.44486587e-04, -3.17874277e-04]), 'sampling_rate': 44100}, 'lyrics': 'Wait wait a minute'}


datasets.arrow_dataset.Dataset

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(timit["train"].remove_columns(["audio"]), num_examples=10)

,lyrics
0,I needed me a die or ride or I need me the one
1,I ain't fucked her yesterday yeah
2,I be discouraged from sharing my blessings
3,I'm out
4,I gave her everything
5,And now they showin' up
6,I can't get you off my mind now
7,Shorty when you riding I be wilding
8,Everyone quick to throw a damn punch
9,I'm just hangin' in the fight


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\*\$\%\"]'

def remove_special_characters(batch):
    from unidecode import unidecode
    batch["lyrics"] = unidecode(batch["lyrics"])
    batch["lyrics"] = re.sub(chars_to_ignore_regex, '', batch["lyrics"]).lower() + " "
    # print(type(batch['lyrics']), batch['lyrics'])
    trans = (
        ('1', 'one'),
        ('2', 'two'),
        ('3', 'three'),
        ('4', 'four'),
        ('5', 'five'),
        ('6', 'six'),
        ('7', 'seven'),
        ('8', 'eight'),
        ('9', 'nine'),
        ('0', 'zero'),
    )

    for num, word in trans:
        batch["lyrics"] = batch["lyrics"].replace(num, word)

    return batch

In [ ]:
timit = timit.map(remove_special_characters)

In [ ]:
show_random_elements(timit["train"].remove_columns(["audio"]))

,lyrics
0,but i ended it by saying i was gonna propose to you
1,jump in the lamb i ain't parkin' it skrr
2,guess we all made for each other
3,can't be out here giving out my pussy 'cause you mad
4,and i'm gettin' rich young nigga bought that coupe ay
5,know you see me through
6,smacking your girlie's box she a camel pack whoo
7,i'm like a exit away yep
8,she think i'm a asshole
9,i'll be your best friend and you'll be my missus


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["lyrics"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = timit.map(
  extract_all_chars,
  batched=True,
  batch_size=-1,
  keep_in_memory=True,
  remove_columns=timit.column_names["train"]
)

Map: 100%|██████████| 3085/3085 [00:00<00:00, 35726.12 examples/s]


In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'s': 0,
 'b': 1,
 'i': 2,
 'o': 3,
 'w': 4,
 'a': 5,
 'e': 6,
 '(': 7,
 'f': 8,
 'm': 9,
 'j': 10,
 'q': 11,
 'd': 12,
 'u': 13,
 'h': 14,
 'n': 15,
 'v': 16,
 'p': 17,
 'l': 18,
 'r': 19,
 "'": 20,
 'x': 21,
 'k': 22,
 'z': 23,
 'y': 24,
 'c': 25,
 ' ': 26,
 'g': 27,
 '/': 28,
 't': 29}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

32

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None

c:\Users\wbsgus\anaconda3\envs\rap\lib\site-packages\transformers\configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  "./",
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.


In [ ]:
model_checkpoint_name = model_checkpoint.split("/")[-1]

### Preprocess Data

In [ ]:
# timit["train"][0]["file"]

In [ ]:
timit["train"][0]["audio"]

{'path': 'E:/rap_rec/dataset_cleaned_sep/train/chunk_0.mp3',
 'array': array([ 0.00000000e+00,  1.46443085e-09,  1.37479139e-09, ...,
        -3.44191722e-05, -3.44486587e-04, -3.17874277e-04]),
 'sampling_rate': 44100}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["lyrics"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=44100)

don't tell me to calm down 


In [ ]:
import librosa
_data = librosa.resample(timit["train"][rand_int]["audio"]["array"], orig_sr=44100, target_sr=16000)
ipd.Audio(data=_data, autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["lyrics"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: ten week i been top tennin' 
Input array shape: (77184,)
Sampling rate: 44100


In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
def prepare_dataset(batch, processor=processor):
    import librosa
    audio = batch["audio"]
    wave = librosa.resample(audio['array'], orig_sr=44100, target_sr=16000)

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(wave, sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["lyrics"]).input_ids
    return batch

In [ ]:
# dir(timit)
timit.column_names

{'train': ['audio', 'lyrics'], 'test': ['audio', 'lyrics']}

In [ ]:
timit = timit.map(prepare_dataset, remove_columns=["audio", "lyrics"], num_proc=4)

Map (num_proc=4): 100%|██████████| 3085/3085 [00:08<00:00, 376.68 examples/s] 


In [ ]:
max_input_length_in_sec = 4.0
min_input_length_in_sec = 1.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
timit["train"] = timit["train"].filter(lambda x: x >= min_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
timit["test"] = timit["test"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
timit["test"] = timit["test"].filter(lambda x: x >= min_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter: 100%|██████████| 2915/2915 [00:00<00:00, 170533.46 examples/s]


## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:



### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

C:\Users\wbsgus\AppData\Local\Temp\ipykernel_19604\24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    lyrics_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=lyrics_str)

    return {"wer": wer}

In [ ]:
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained(
    model_checkpoint,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

c:\Users\wbsgus\anaconda3\envs\rap\lib\site-packages\transformers\configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='output',
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  # num_train_epochs=30,
  num_train_epochs=20,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  # learning_rate=1e-5,
  weight_decay=0.001,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

### Training

In [ ]:
import warnings

warnings.filterwarnings(action='ignore', module='torch')
warnings.filterwarnings(action='ignore', module='transformers')
trainer.train()

  8%|▊         | 500/6560 [07:44<42:08,  2.40it/s]  

{'loss': 3.8355, 'learning_rate': 4.93e-05, 'epoch': 1.52}


                                                  
  8%|▊         | 500/6560 [08:20<42:08,  2.40it/s]

{'eval_loss': 3.014528751373291, 'eval_wer': 1.0, 'eval_runtime': 36.4003, 'eval_samples_per_second': 66.181, 'eval_steps_per_second': 8.297, 'epoch': 1.52}


 15%|█▌        | 1000/6560 [11:35<41:20,  2.24it/s]  

...

                                                   
 99%|█████████▉| 6500/6560 [53:46<00:25,  2.32it/s]

{'eval_loss': 2.0230443477630615, 'eval_wer': 0.7849411517444305, 'eval_runtime': 32.6548, 'eval_samples_per_second': 73.772, 'eval_steps_per_second': 9.248, 'epoch': 19.82}


100%|██████████| 6560/6560 [54:08<00:00,  2.02it/s]

{'train_runtime': 3248.3488, 'train_samples_per_second': 64.531, 'train_steps_per_second': 2.019, 'train_loss': 1.615396383913552, 'epoch': 20.0}


TrainOutput(global_step=6560, training_loss=1.615396383913552, metrics={'train_runtime': 3248.3488, 'train_samples_per_second': 64.531, 'train_steps_per_second': 2.019, 'train_loss': 1.615396383913552, 'epoch': 20.0})